# Toronto Neighborhoods - PART 1

In [35]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Initialize BeautifulSoup and download text from Wikipedia

In [36]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Alter the data and convert it to dataframe

In [37]:
# Initialize variables
first = []
pc = []
bo = []
ne = []

# Find the table and scrape content
table = soup.find('table', class_='wikitable sortable')
assigned = table.find_all('td')

for row in table.find_all('td'):
    first.append(row.text)
    
# Iterate through data and save it to 3 separate lists
rows = len(first)//3

for i in range(0,rows,3):
    pc.append(first[i])
for i in range(1,rows,3):
    bo.append(first[i])
for i in range(2,rows,3):
    ne.append(first[i].strip())

# Initialize dataframe and fill it with content
df = pd.DataFrame()
df['Postcode'] = pc
df['Borough'] = bo
df['Neighbourhood'] = ne

# Drop rows where 'Borough' is not assigned
df = df[df.Borough != 'Not assigned']

# Group Neighbourhoods with same Postcode
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x.astype(str).unique())).reset_index()

# Assign name to Neighbourhood if 'Not assigned'
count = range(df.shape[0] - 1)

for i in count:
    if (df['Neighbourhood'][i] == 'Not assigned'):
        df['Neighbourhood'][i] = df['Borough'][i]

df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [38]:
df.shape

(41, 3)